### Importing Libraries

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch.nn as nn

# set gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# set all seed
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed = 0
set_seed(seed)

Using device: cuda


### Defining Important Functions

In [2]:

def plot_data(df, x_col, y_col, title, xlabel, ylabel, figsize=(15,5)):
    plt.figure(figsize=figsize)
    plt.plot(df[x_col], df[y_col])
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid()
    plt.show()

def calculate_nse(observed, predicted):
    """Calculate Nash-Sutcliffe Efficiency (NSE)"""
    numerator = np.sum((observed - predicted) ** 2)
    denominator = np.sum((observed - np.mean(observed)) ** 2)
    nse = 1 - (numerator / denominator)
    return nse

def calculate_rmse(observed, predicted):
    """Calculate Root Mean Square Error (RMSE)"""
    rmse = np.sqrt(np.mean((observed - predicted) ** 2))
    return rmse


def calculate_mae(observed, predicted):
    """Calculate Mean Absolute Error (MAE)"""
    mae = np.mean(np.abs(observed - predicted))
    return mae

def calculate_physical_consistency(observed, predicted):
    """Check if predicted values are realistic (e.g., non-negative)"""
    consistency = np.all(predicted >= 0)
    return consistency

# Calculate performance metrics
def calc_metrics(observed, predicted):
    rmse = calculate_rmse(observed, predicted)
    mae = calculate_mae(observed, predicted)
    nse = calculate_nse(observed, predicted)
    physical_consistency = calculate_physical_consistency(observed, predicted)
    print(f"RMSE = {rmse:.2f}, MAE = {mae:.2f}, NSE = {nse:.2f}, Physical Consistency = {physical_consistency}")
    return rmse, mae, nse, physical_consistency


def plot_ground_truth_vs_prediction(df, y_pred, title_hourly, title_daily, ylabel, figsize=(15, 5)):
    """Plot ground truth vs predictions for hourly and daily data."""
    # Plot hourly predictions
    plt.figure(figsize=figsize)
    plt.plot(df['valid_time'], df['tp'], label='Ground Truth', color='blue')
    plt.plot(df['valid_time'], y_pred[:, 0], label='Predicted Hourly', color='orange')
    plt.title(title_hourly)
    plt.xlabel('Date')
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid()
    plt.show()

    # Plot daily predictions
    daily_pred_df = pd.merge(df[['valid_time']], pd.DataFrame(y_pred, columns=target), left_index=True, right_index=True)
    daily_pred_df = daily_pred_df[daily_pred_df['valid_time'].dt.hour == 0]
    plt.figure(figsize=figsize)
    plt.plot(df[df['valid_time'].dt.hour == 0]['valid_time'], df[df['valid_time'].dt.hour == 0]['tp'], label='Ground Truth', color='blue')
    plt.plot(daily_pred_df['valid_time'], daily_pred_df['tp'], label='Predicted Daily', color='orange')
    plt.title(title_daily)
    plt.xlabel('Date')
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid()
    plt.show()


### Loading Data

In [3]:
def add_cycle_features(df):
    """Add cyclic features for month and year to the dataframe."""
    df['valid_time'] = pd.to_datetime(df['valid_time'])
    df['month_sin'] = np.sin(2 * np.pi * df['valid_time'].dt.month / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['valid_time'].dt.month / 12)
    df['year_sin'] = np.sin(2 * np.pi * df['valid_time'].dt.year / 12)
    df['year_cos'] = np.cos(2 * np.pi * df['valid_time'].dt.year / 12)
    return df

In [4]:
data_dir = './data/'

train_df = pd.read_csv(os.path.join(data_dir, 'train_data.csv'))
train_df = add_cycle_features(train_df)
train_df.drop(columns=['second', 'minute'], inplace=True)
train_df['valid_time'] = pd.to_datetime(train_df['valid_time'])
print("Shape of training data:", train_df.shape)

val_df = pd.read_csv(os.path.join(data_dir, 'val_data.csv'))
val_df = add_cycle_features(val_df)
val_df.drop(columns=['second', 'minute'], inplace=True)
val_df['valid_time'] = pd.to_datetime(val_df['valid_time'])
print("Shape of validation data:", val_df.shape)

test_df = pd.read_csv(os.path.join(data_dir, 'test_data.csv'))
test_df = add_cycle_features(test_df)
test_df.drop(columns=['second', 'minute'], inplace=True)
test_df['valid_time'] = pd.to_datetime(test_df['valid_time'])
print("Shape of test data:", test_df.shape)

Shape of training data: (87672, 25)
Shape of validation data: (17520, 25)
Shape of test data: (17544, 25)


In [5]:
display(train_df.head())


,valid_time,tp,year,month,day,hour,yearly_mean,yearly_max,yearly_min,yearly_std,...,monthly_var,daily_mean,daily_max,daily_min,daily_std,daily_var,month_sin,month_cos,year_sin,year_cos
0,2011-01-01 00:00:00,0.000166,2011,1,1,0,0.00028,0.003323,0.0,0.000407,...,9.706095e-10,0.000069,0.000166,0.000026,0.000042,1.801785e-09,0.5,0.866025,-0.5,-0.866025
1,2011-01-01 01:00:00,0.000103,2011,1,1,1,0.00028,0.003323,0.0,0.000407,...,9.706095e-10,0.000069,0.000166,0.000026,0.000042,1.801785e-09,0.5,0.866025,-0.5,-0.866025
2,2011-01-01 02:00:00,0.000083,2011,1,1,2,0.00028,0.003323,0.0,0.000407,...,9.706095e-10,0.000069,0.000166,0.000026,0.000042,1.801785e-09,0.5,0.866025,-0.5,-0.866025
3,2011-01-01 03:00:00,0.000078,2011,1,1,3,0.00028,0.003323,0.0,0.000407,...,9.706095e-10,0.000069,0.000166,0.000026,0.000042,1.801785e-09,0.5,0.866025,-0.5,-0.866025
4,2011-01-01 04:00:00,0.000095,2011,1,1,4,0.00028,0.003323,0.0,0.000407,...,9.706095e-10,0.000069,0.000166,0.000026,0.000042,1.801785e-09,0.5,0.866025,-0.5,-0.866025


In [ ]:
features = ['year', 'month', 'day', 'hour',
            'yearly_mean', 'yearly_max', 'yearly_min', 'yearly_std', 'yearly_var',
            'monthly_mean', 'monthly_max', 'monthly_min', 'monthly_std', 'monthly_var',
            'month_sin', 'month_cos', 'year_sin', 'year_cos']
target = ['tp']


for df in [train_df, val_df, test_df]:
    for feature in [f for f in features + target if f not in ['month_sin', 'month_cos', 'year_sin', 'year_cos', 'year', 'month', 'day', 'hour']]:
        df[feature] = np.log(df[feature] + 1e-4)  # Avoid log(0) by adding a small constant

## Training

In [7]:
class TimeSeriesDataset(Dataset):
    def __init__(self, df, features, target, seq_length=24):
        self.X = df[features].values.astype('float32')
        self.y = df[target].values.astype('float32')
        self.seq_length = seq_length

    def __len__(self):
        return len(self.X) - self.seq_length + 1

    def __getitem__(self, idx):
        X = self.X[idx:idx + self.seq_length]
        y = self.y[idx + self.seq_length - 1]
        return torch.tensor(X), torch.tensor(y)

seq_length = 24*2

# Create datasets
train_dataset = TimeSeriesDataset(train_df, features, target, seq_length)
val_dataset = TimeSeriesDataset(val_df, features, target, seq_length)
test_dataset = TimeSeriesDataset(test_df, features, target, seq_length)

batch_size = 24

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=24, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=24, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=24, shuffle=False)


In [8]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initialize hidden state and cell state (batch_size, num_layers, hidden_size)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # x shape: (batch_size, seq_length, input_size)
        out, _ = self.lstm(x, (h0, c0))

        # Apply fully connected layer to the last time step
        out = self.fc(out[:, -1, :])
        return out

In [9]:
# Hyperparameters (adjust based on your data)
input_size = len(features)  # Number of features
hidden_size = 64  # Number of LSTM units
num_layers = 2
output_size = len(target)  # Number of target variables
learning_rate = 0.001

save_path = './models/'
import shutil
if os.path.exists(save_path):
    shutil.rmtree(save_path)
os.makedirs(save_path, exist_ok=True)

Input size: 18
Hidden size: 64
Number of layers: 2
Output size: 1


In [10]:
model = LSTMModel(input_size, hidden_size, num_layers, output_size).to(device)
print(model)

LSTMModel(
  (lstm): LSTM(18, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)


In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, save_checkpoint=True):
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    best_model = None

    for epoch in range(num_epochs):
        model.train()
        epoch_train_loss = 0.0
        for batch_idx, (X, y) in tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
            X = X.to(device)
            y = y.to(device)

            # Forward pass
            outputs = model(X)
            loss = criterion(outputs, y)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_train_loss += loss.item() * X.size(0)
        epoch_train_loss /= len(train_loader.dataset)
        train_losses.append(epoch_train_loss)
        
        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch_idx, (X, y) in tqdm(enumerate(val_loader), total=len(val_loader), desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
                X = X.to(device)
                y = y.to(device)

                outputs = model(X)
                loss = criterion(outputs, y)
                val_loss += loss.item() * X.size(0)
        val_loss /= len(val_loader.dataset)
        val_losses.append(val_loss)

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_train_loss:.4f}, Val Loss: {val_loss:.4f}')

        # Save the best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model.state_dict()
            torch.save(model.state_dict(), save_path + 'best_model.pth')
            print(f"Best model saved with validation loss: {best_val_loss:.4f}")
        
        if save_checkpoint:
            torch.save(model.state_dict(), save_path + f'model_epoch_{epoch+1}.pth')

    return model, best_model, train_losses, val_losses


def evaluate_model(model, test_loader):
    model.eval()
    test_loss = 0.0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for batch_idx, (X, y) in tqdm(enumerate(test_loader), total=len(test_loader)):
            X = X.to(device)
            y = y.to(device)
            outputs = model(X)
            loss = criterion(outputs, y)
            test_loss += loss.item() * X.size(0)
            all_preds.append(outputs.cpu().numpy())
            all_targets.append(y.cpu().numpy())
    test_loss /= len(test_loader.dataset)
    print(f'Test Loss: {test_loss:.4f}')

    all_preds = np.concatenate(all_preds, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)
    all_preds = np.exp(all_preds) - 1e-4  # Inverse transformation
    all_targets = np.exp(all_targets) - 1e-4  # Inverse transformation
    return all_preds, all_targets, test_loss

def plot_loss(train_losses, val_losses):
    """Plot training and validation loss."""
    plt.figure(figsize=(15, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
num_epochs = 20

# Train the model
model, best_model, train_losses, val_losses = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)


In [ ]:
plot_loss(train_losses, val_losses)

In [ ]:
# Load the best model
model.load_state_dict(best_model)
model.eval()
# Evaluate the model on the test set
y_pred, y_true, test_loss = evaluate_model(model, test_loader)
# Calculate performance metrics
rmse, mae, nse, physical_consistency = calc_metrics(y_true, y_pred)
# Plot ground truth vs predictions
plt.figure(figsize=(15, 5))
plt.plot(test_df['valid_time'], test_df['tp'], label='Ground Truth', color='blue')
plt.plot(test_df['valid_time'], y_pred[:, 0], label='Predicted', color='orange')
plt.title('Ground Truth vs Predictions')
plt.xlabel('Date')
plt.ylabel('Precipitation (mm)')
plt.legend()
plt.grid()
plt.show()